# Part II — Core Django  
## 7. Templates (Django Templating Language) — Comprehensive Workbook Chapter

Django templates are how you generate **HTML** (and sometimes plain text like email
templates) in a safe, maintainable way.

This chapter will take your project from “views return strings/JSON” to an
industry-standard server-rendered site structure:

- `base.html` layout + page inheritance
- consistent navigation using named URLs
- reusable partials (`include`)
- safe variable rendering (escaping/XSS prevention)
- static assets (CSS)
- consistent template organization (`app/templates/app/...`)
- custom filters/tags (so templates stay clean)

---

## 7.0 Learning Outcomes

By the end of this chapter you should be able to:

1. Explain what a template engine is and why Django templates exist.
2. Explain how Django finds templates:
   - `TEMPLATES[...]["DIRS"]`
   - `APP_DIRS=True` and app template folders
3. Render templates correctly using `render(request, template_name, context)`.
4. Use Django Template Language (DTL):
   - variables `{{ ... }}`
   - tags `{% ... %}`
   - comments `{# ... #}`
   - filters `{{ value|filter }}`
5. Build a proper base layout with inheritance (`{% extends %}`, `{% block %}`).
6. Use `{% url %}` everywhere instead of hardcoded links.
7. Use `{% include %}` for reusable partials (nav, footer, etc.).
8. Understand escaping, autoescaping, and how XSS happens.
9. Add static files and reference them with `{% load static %}` and `{% static %}`.
10. Create a custom template filter and explain why it’s better than template logic.
11. Write tests that verify templates render correctly (`assertTemplateUsed`,
    `assertContains`).

---

## 7.1 What Templates Are (And Why Django Doesn’t Want You Writing HTML in Views)

### 7.1.1 The problem templates solve
If you build HTML in Python strings:

- it becomes unreadable quickly
- mixing presentation with business logic becomes messy
- repeated layout (nav, footer) gets duplicated everywhere
- maintaining consistent UI is painful

Templates separate concerns:

- **view**: decide *what data* to show and *which template* to use
- **template**: decide *how to present* that data as HTML

### 7.1.2 Template engines in plain terms
A template engine takes:
- a template file (HTML + placeholders)
- a context (dictionary-like data)
and produces:
- rendered HTML (a string)

Conceptual:

```text
template.html + {"name": "Asha"}  -->  "<h1>Hello Asha</h1>"
```

---

## 7.2 How Django Finds Templates (The Source of Most “TemplateDoesNotExist” Errors)

Django template discovery is controlled by the `TEMPLATES` setting in
`config/settings.py`.

Typical generated setting (simplified):

```python
TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [],
        "APP_DIRS": True,
        "OPTIONS": {
            "context_processors": [
                ...
            ],
        },
    },
]
```

### 7.2.1 `APP_DIRS=True`: app templates discovery
When `APP_DIRS=True`, Django will look inside each installed app for templates in
this pattern:

```text
<app_name>/templates/...
```

Industry best practice is to namespace inside the app:

```text
pages/templates/pages/home.html
articles/templates/articles/list.html
```

Why namespace?
Because two apps might have `home.html`. If you don’t namespace, you get collisions
and unpredictable template selection based on app order.

**Good (namespaced):**
- `pages/home.html`
- `articles/home.html`

**Risky (not namespaced):**
- `home.html` in multiple apps

### 7.2.2 `DIRS`: project-level templates directory
`DIRS` is for templates that are not tied to a single app, typically:
- `base.html`
- shared partials like `partials/nav.html`
- error pages like `404.html` (project-wide)

Industry standard approach:
- put site-wide templates in a top-level `templates/` folder
- include it in `DIRS`

---

## 7.3 Configure a Project-Level `templates/` Directory (Industry Standard)

### 7.3.1 Create directory structure
At repo root (same level as `manage.py`), create:

```text
templates/
  base.html
  partials/
    _nav.html
    _footer.html
```

### 7.3.2 Update `TEMPLATES["DIRS"]`
In `config/settings.py`, find `BASE_DIR` (generated by Django) and update `DIRS`.

Typically you already have:

```python
from pathlib import Path

BASE_DIR = Path(__file__).resolve().parent.parent
```

Set `DIRS` like:

```python
TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [BASE_DIR / "templates"],
        "APP_DIRS": True,
        "OPTIONS": {
            "context_processors": [
                "django.template.context_processors.debug",
                "django.template.context_processors.request",
                "django.contrib.auth.context_processors.auth",
                "django.contrib.messages.context_processors.messages",
            ],
        },
    },
]
```

#### Explain what this changes
Now Django will search templates in (high-level order):
1. `BASE_DIR/templates/` (because you added it to `DIRS`)
2. each installed app’s `templates/` directory (because `APP_DIRS=True`)

If you have `base.html` in `BASE_DIR/templates/`, it will always be found even if an
app accidentally has another `base.html`.

---

## 7.4 Django Template Language (DTL): Syntax You Must Master

DTL has three main syntaxes:

### 7.4.1 Variables: `{{ ... }}`
Variables print data into HTML.

Example:

```django
<h1>Hello {{ name }}</h1>
```

If context is `{"name": "Asha"}`, output is:

```html
<h1>Hello Asha</h1>
```

### 7.4.2 Tags: `{% ... %}`
Tags do logic/control flow and template structure:

- `{% if %}`, `{% for %}`
- `{% extends %}`, `{% block %}`
- `{% url %}`, `{% include %}`, `{% load static %}`

Example:

```django
{% if user.is_authenticated %}
  <p>Welcome back, {{ user.username }}</p>
{% else %}
  <p>Please log in</p>
{% endif %}
```

### 7.4.3 Comments: `{# ... #}`
Comments are removed from output:

```django
{# This won't appear in HTML #}
```

---

## 7.5 Rendering Your First Real Template (Convert `pages.home`)

Right now your `pages.home` returns `HttpResponse("...")`. We’ll convert it to a
proper template render.

### 7.5.1 Create `pages/templates/pages/home.html`
Create directories:

```text
pages/
  templates/
    pages/
      home.html
```

Create `pages/templates/pages/home.html`:

```django
{% extends "base.html" %}

{% block title %}Home{% endblock %}

{% block content %}
  <h1>Home</h1>

  <p>This page is rendered by Django templates.</p>

  <h2>Template Context Demo</h2>
  <ul>
    <li>Server says: {{ message }}</li>
    <li>Method: {{ request.method }}</li>
    <li>Path: {{ request.path }}</li>
  </ul>
{% endblock %}
```

#### Explain what’s happening
- `{% extends "base.html" %}` means:
  - “Use base.html as the outer layout.”
  - This template only fills in specific blocks.
- `{{ request.method }}` and `{{ request.path }}` work because you have the
  `request` context processor enabled:
  - `"django.template.context_processors.request"`

### 7.5.2 Create `templates/base.html`
Create `templates/base.html`:

```django
{% load static %}

<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />

    <title>{% block title %}Site{% endblock %}</title>

    <link rel="stylesheet" href="{% static 'css/site.css' %}" />
  </head>
  <body>
    {% include "partials/_nav.html" %}

    <main class="container">
      {% block content %}{% endblock %}
    </main>

    {% include "partials/_footer.html" %}
  </body>
</html>
```

#### Explain the major pieces
- `{% load static %}` loads the static template tag library so `{% static ... %}`
  works.
- `{% block title %}` is a placeholder that child templates override.
- `{% block content %}` is where page-specific HTML will go.
- `{% include %}` inserts another template’s rendered output. This is how you avoid
  duplicating nav/footer across pages.

### 7.5.3 Create partials
Create `templates/partials/_nav.html`:

```django
<nav class="nav">
  <a href="{% url 'pages:home' %}">Home</a>
  <a href="{% url 'articles:list_html' %}">Articles</a>
  <a href="{% url 'pages:healthz' %}">Health</a>
</nav>
```

Create `templates/partials/_footer.html`:

```django
<footer class="footer">
  <small>Rendered by Django • Path: {{ request.path }}</small>
</footer>
```

#### Why these partials are “industry standard”
- Nav/footer appear on every page.
- If you hardcode them in each template, you create duplication and inconsistency.
- Partials make updates one-file changes.

---

## 7.6 Update the View to Use `render()`

Edit `pages/views.py`:

```python
from django.http import JsonResponse
from django.shortcuts import render


def home(request):
    return render(
        request,
        "pages/home.html",
        {
            "message": "Hello from Django templates",
        },
    )


def healthz(request):
    return JsonResponse({"status": "ok"})
```

### Explain `render()` precisely
`render(request, template_name, context)`:

1. Loads the template named `"pages/home.html"` using template loaders.
2. Combines the context dict with:
   - context processors (like `request`, `user`, `messages`)
3. Renders final HTML.
4. Returns an `HttpResponse` with that HTML.

**Why passing `request` matters:**  
It enables context processors and features like CSRF token generation.

---

## 7.7 Static Files (CSS): Make the Site Look Like Something (and Learn the Pipeline)

### 7.7.1 What static files are in Django terms
Static files are assets shipped with your code:
- CSS
- JS
- images that are part of the UI

In development:
- Django can serve them automatically if `django.contrib.staticfiles` is enabled
  (it is by default).

In production:
- static files are collected and served by a real web server/CDN (later chapters).

### 7.7.2 Create a CSS file
Create:

```text
static/
  css/
    site.css
```

Create `static/css/site.css`:

```css
.container {
  max-width: 900px;
  margin: 2rem auto;
  padding: 0 1rem;
  font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif;
}

.nav {
  display: flex;
  gap: 1rem;
  padding: 1rem;
  background: #f3f3f3;
}

.footer {
  margin-top: 3rem;
  padding: 1rem;
  background: #f9f9f9;
  color: #444;
}
```

### 7.7.3 Why `{% static 'css/site.css' %}` looks like that
- `static` is a template tag that builds the correct URL for the asset.
- The string `'css/site.css'` is the path inside your static files directories.
- Django outputs something like:

```html
<link rel="stylesheet" href="/static/css/site.css" />
```

Using `{% static %}` is critical because:
- the static URL prefix might change in production
- CDNs may rewrite or version static paths later
- you avoid hardcoding `/static/` everywhere

---

## 7.8 Template Inheritance: The Most Important Template Pattern

### 7.8.1 Why inheritance exists
Most pages share:
- `<html>`, `<head>`, CSS links
- navigation
- consistent layout

Only the content changes.

Inheritance ensures:
- one source of truth for layout
- consistent site-wide changes
- fewer bugs

### 7.8.2 Blocks: how child templates “fill in” areas
In `base.html`, you defined:

```django
<title>{% block title %}Site{% endblock %}</title>
...
{% block content %}{% endblock %}
```

In child templates (like `pages/home.html`), you override:

```django
{% block title %}Home{% endblock %}
{% block content %} ... {% endblock %}
```

### 7.8.3 `block.super` (advanced but very practical)
If base template sets some default and you want to add to it, use `block.super`.

Example:

```django
{% block title %}{{ block.super }} • Home{% endblock %}
```

This takes whatever base block produced and extends it.

---

## 7.9 Loops, Conditionals, and Filters (With Correct “Do’s and Don’ts”)

### 7.9.1 Loops: `{% for %}`
Example list rendering:

```django
<ul>
  {% for item in items %}
    <li>{{ item }}</li>
  {% empty %}
    <li>No items found.</li>
  {% endfor %}
</ul>
```

Key points:
- `{% empty %}` is a clean pattern to handle empty lists.
- Avoid complex computations inside templates; do it in Python.

### 7.9.2 Conditionals: `{% if %}`
Example:

```django
{% if is_feature_enabled %}
  <p>Feature enabled.</p>
{% else %}
  <p>Feature disabled.</p>
{% endif %}
```

### 7.9.3 Filters: `{{ value|filter }}`
Filters transform values for display.

Examples you’ll use constantly:

- default when missing:

```django
<p>Nickname: {{ user.nickname|default:"(none)" }}</p>
```

- truncate long text:

```django
<p>{{ article.body|truncatechars:120 }}</p>
```

- date formatting:

```django
<p>Published: {{ article.published_at|date:"Y-m-d H:i" }}</p>
```

Why filters matter:
- display formatting should be in templates
- business logic should not be in templates
- filters keep templates readable

---

## 7.10 Escaping and XSS (Why Django Templates Are “Safe by Default”)

### 7.10.1 Autoescaping: the default behavior
By default, `{{ variable }}` is **escaped** for HTML.

If `name = "<script>alert(1)</script>"`, then:

```django
<p>{{ name }}</p>
```

renders as:

```html
<p>&lt;script&gt;alert(1)&lt;/script&gt;</p>
```

So the script does not execute.

### 7.10.2 How you accidentally break safety
If you do:

```django
<p>{{ name|safe }}</p>
```

you tell Django:
- “Trust this content as HTML.”

Now the browser may execute it as code.

**Industry rule:** only use `safe` when:
- content is trusted, or
- content has been sanitized by a robust HTML sanitizer

### 7.10.3 Escaping in different contexts (important nuance)
Escaping rules differ for:
- HTML body
- HTML attributes
- JavaScript strings
- CSS
- URLs

Django’s default escaping is mainly for HTML contexts. If you embed untrusted data
into JS directly, you can still create vulnerabilities.

Safe approach:
- keep untrusted content in normal HTML text nodes (`{{ ... }}`)
- avoid generating JS code with untrusted strings inside templates

We’ll revisit this in the security chapter with concrete cases.

---

## 7.11 LAB: Build an HTML Articles Page Using Templates (Not JSON)

You currently have JSON endpoints for articles. Now we’ll create an HTML list page
to practice templates.

### 7.11.1 Add an HTML list view to `articles/views.py`

Edit `articles/views.py` (keep your JSON list/detail, add HTML list):

```python
from django.http import Http404, JsonResponse
from django.shortcuts import render
from django.views.decorators.http import require_GET

from .data import ARTICLES


@require_GET
def article_list(request):
    tag = request.GET.get("tag")
    q = request.GET.get("q")

    results = ARTICLES

    if tag:
        results = [a for a in results if tag in a["tags"]]

    if q:
        q_lower = q.strip().lower()
        results = [
            a
            for a in results
            if q_lower in a["title"].lower() or q_lower in a["body"].lower()
        ]

    return JsonResponse({"count": len(results), "results": results})


@require_GET
def article_detail(request, slug: str):
    for a in ARTICLES:
        if a["slug"] == slug:
            return JsonResponse(a)
    raise Http404("Article not found")


@require_GET
def article_list_html(request):
    tag = request.GET.get("tag")
    q = request.GET.get("q")

    results = ARTICLES
    if tag:
        results = [a for a in results if tag in a["tags"]]
    if q:
        q_lower = q.strip().lower()
        results = [
            a
            for a in results
            if q_lower in a["title"].lower() or q_lower in a["body"].lower()
        ]

    return render(
        request,
        "articles/list.html",
        {
            "articles": results,
            "tag": tag,
            "q": q,
        },
    )
```

#### Why we still parse `tag` and `q` here
Templates should not decide which articles match filters. The view decides and hands
the final list to the template. This keeps templates simple and testable.

### 7.11.2 Wire the HTML list URL
Edit `articles/urls.py`:

```python
from django.urls import path

from . import views

app_name = "articles"

urlpatterns = [
    path("", views.article_list, name="list"),
    path("html/", views.article_list_html, name="list_html"),
    path("<slug:slug>/", views.article_detail, name="detail"),
]
```

Now:
- `/articles/` returns JSON list (existing)
- `/articles/html/` returns HTML list page (new)

### 7.11.3 Create the template `articles/templates/articles/list.html`

Create:

```text
articles/
  templates/
    articles/
      list.html
```

Create `articles/templates/articles/list.html`:

```django
{% extends "base.html" %}

{% block title %}Articles{% endblock %}

{% block content %}
  <h1>Articles</h1>

  <section>
    <h2>Current Filters</h2>
    <ul>
      <li>tag: {{ tag|default:"(none)" }}</li>
      <li>q: {{ q|default:"(none)" }}</li>
    </ul>
  </section>

  <section>
    <h2>Results ({{ articles|length }})</h2>

    <ul>
      {% for a in articles %}
        <li>
          <strong>{{ a.title }}</strong>
          <div>
            <small>slug: {{ a.slug }}</small>
          </div>
          <div>
            tags:
            {% for t in a.tags %}
              <code>{{ t }}</code>
            {% endfor %}
          </div>
          <p>{{ a.body|truncatechars:80 }}</p>
        </li>
      {% empty %}
        <li>No articles matched your filters.</li>
      {% endfor %}
    </ul>
  </section>
{% endblock %}
```

#### Explain key template features used here
- `{{ tag|default:"(none)" }}` ensures missing query params don’t show as empty.
- `{{ articles|length }}` shows how many items are in the list.
- Nested `{% for %}` loop shows tags per article.
- `truncatechars` is a display-only formatting tool (good use of templates).

### 7.11.4 Try it
Run server:

```bash
python manage.py runserver
```

Visit:
- `http://127.0.0.1:8000/articles/html/`
- `http://127.0.0.1:8000/articles/html/?tag=django`
- `http://127.0.0.1:8000/articles/html/?q=views`

---

## 7.12 Context Processors (How `request`, `user`, and `messages` Appear in Templates)

A context processor is a function that injects variables into every template
rendered with a `RequestContext` (which `render()` uses automatically).

You already have these common ones:

- `request` → gives `request` object in templates
- `auth` → gives `user`
- `messages` → gives flash messages (later)

### 7.12.1 Add your own context processor (site-wide variables)

Create `config/context_processors.py`:

```python
def site_info(request):
    return {
        "SITE_NAME": "Django Mastery Workbook",
    }
```

Add it to `TEMPLATES` in `config/settings.py`:

```python
"context_processors": [
    "django.template.context_processors.debug",
    "django.template.context_processors.request",
    "django.contrib.auth.context_processors.auth",
    "django.contrib.messages.context_processors.messages",
    "config.context_processors.site_info",
],
```

Now update `templates/base.html` title:

```django
<title>
  {% block title %}{{ SITE_NAME }}{% endblock %}
</title>
```

And maybe show it in nav:

```django
<nav class="nav">
  <strong>{{ SITE_NAME }}</strong>
  <a href="{% url 'pages:home' %}">Home</a>
  <a href="{% url 'articles:list_html' %}">Articles</a>
</nav>
```

#### Why context processors are useful
- avoid repeating `"SITE_NAME"` in every view context
- centralize “global context”
- keep view contexts focused on page-specific data

**Industry warning:** don’t put expensive DB queries in context processors. They run
for every request that renders templates.

---

## 7.13 Custom Template Filters (Keep Templates Clean and Reusable)

If your template starts doing messy formatting, it’s usually a sign you should create
a filter/tag.

### 7.13.1 Create a custom filter: `excerpt`
Goal: convert long text into a safe excerpt, consistently.

Create:

```text
pages/
  templatetags/
    __init__.py
    text_extras.py
```

Create `pages/templatetags/text_extras.py`:

```python
from django import template

register = template.Library()


@register.filter
def excerpt(value: str, max_len: int = 120) -> str:
    if value is None:
        return ""

    text = str(value).strip()
    if len(text) <= max_len:
        return text

    return text[: max_len - 1].rstrip() + "…"
```

#### Explain what’s going on
- `templatetags/` is a special Django discovery package name.
- `register = template.Library()` is required to register filters/tags.
- `@register.filter` exposes `excerpt` to templates.

### 7.13.2 Use it in the articles template
In `articles/templates/articles/list.html`, load it and use it:

```django
{% extends "base.html" %}
{% load text_extras %}

...

<p>{{ a.body|excerpt:80 }}</p>
```

#### Why this is better than `truncatechars` sometimes
- you can enforce a consistent style (ellipsis, stripping, handling None)
- you can add more logic later (word boundaries, etc.) without changing templates
- you avoid copy-pasting formatting logic across templates

---

## 7.14 Template Testing (Prove Pages Render Correctly)

Testing templates isn’t about “pixel-perfect HTML.” It’s about verifying:
- correct status code
- correct template used
- critical content is present
- view passes expected context values

### 7.14.1 Add tests for home template
Edit `pages/tests.py` (add to your existing tests):

```python
from django.test import TestCase
from django.urls import reverse


class PagesTemplateTests(TestCase):
    def test_home_uses_template(self):
        url = reverse("pages:home")
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
        self.assertTemplateUsed(response, "pages/home.html")
        self.assertContains(response, "This page is rendered by Django templates.")
```

### 7.14.2 Add tests for articles HTML list
Create or edit `articles/tests.py`:

```python
from django.test import TestCase
from django.urls import reverse


class ArticlesHtmlTests(TestCase):
    def test_articles_html_page_renders(self):
        url = reverse("articles:list_html")
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
        self.assertTemplateUsed(response, "articles/list.html")
        self.assertContains(response, "<h1>Articles</h1>", html=True)

    def test_articles_html_filtering(self):
        url = reverse("articles:list_html")
        response = self.client.get(url, {"tag": "cbv"})
        self.assertEqual(response.status_code, 200)

        # We expect the CBV article to appear:
        self.assertContains(response, "Class-Based Views")
```

#### Why `assertTemplateUsed` is valuable
If someone refactors and accidentally changes a view to render the wrong template,
tests catch it immediately.

---

## 7.15 Common Template Mistakes (And Their Fixes)

### Mistake A: `TemplateDoesNotExist`
Typical causes:
- file placed in wrong folder
- missing `APP_DIRS=True`
- app not in `INSTALLED_APPS`
- template referenced with wrong name

Fix checklist:
1. If app template: ensure path is `app/templates/app/template.html`
2. Ensure app is in `INSTALLED_APPS`
3. Ensure `APP_DIRS=True`
4. Ensure `DIRS` includes project templates path (if using `templates/`)

### Mistake B: Hardcoded links (`href="/articles/html/"`)
Fix:
- always use `{% url 'namespace:name' %}`

### Mistake C: Too much logic in templates
If your template contains complex nested conditions and computations:
- move logic to view or a custom filter/tag
- pass clean, prepared data into context

### Mistake D: Using `safe` on user input
Fix:
- remove `safe`
- sanitize HTML if you truly need rich text (advanced topic)

---

## 7.16 Exercises (Do These Before Proceeding)

1. Add `pages/templates/pages/about.html`:
   - extends base
   - has a title block and content block
   - add route `/about/` using `TemplateView` or FBV with `render()`

2. Update nav partial:
   - add link to About using `{% url 'pages:about' %}`
   - ensure no hardcoded paths remain

3. Add an “active link” indicator:
   - if `request.path` equals the link path, add a CSS class
   - keep logic minimal; you can do:

```django
{% url 'pages:home' as home_url %}
<a class="{% if request.path == home_url %}active{% endif %}" href="{{ home_url }}">
  Home
</a>
```

4. Create one more custom filter:
   - `upper_first` that converts `"hello world"` → `"Hello world"`
   - use it in at least one template

5. Run:
```bash
python manage.py test
python -m ruff check .
python -m black . --check
```

---

## 7.17 Chapter Summary

- Templates turn context data into HTML safely and consistently.
- Django finds templates via:
  - project `DIRS` (site-wide templates)
  - app templates (`APP_DIRS=True`)
- Use inheritance + includes to eliminate duplication (`base.html`, partials).
- Use `{% url %}` for maintainable links; avoid hardcoded paths.
- Autoescaping prevents many XSS bugs—don’t disable it casually.
- Custom filters/tags keep templates clean and business logic out of HTML.
- Tests should verify template usage and critical content.

---

Next chapter: **Part II — Chapter 8: Models and Database Basics**  
We’ll create real database models, migrations, relationships, and start building
CRUD pages the way professional Django apps do.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. url_routing_and_views.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='8. models_and_database_basics.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
